# Phân tích chi tiết file vector `.pt` và in thống kê (Inspect Torch Tensor)

## Mục tiêu của script
Script này dùng để:
- **Đọc một file `.pt`** đã được lưu bằng `torch.save()`
- Xác định dữ liệu trong file là:
  - **`dict`** (nhiều key → nhiều tensor) hoặc
  - **tensor đơn**
- In thông tin chi tiết cho từng tensor:
  - `shape`, `dtype`
  - thống kê cơ bản: `min`, `max`, `mean`, `std`
  - **L2 norm** (độ dài vector)
  - dữ liệu mẫu (head/tail)
  - kiểm tra **sparsity** (tỉ lệ phần tử bằng 0)
- Nếu tensor là `token_weights`:
  - in toàn bộ trọng số token
  - in tổng trọng số để kiểm tra

## Thư viện sử dụng
- `torch`: load file `.pt`, kiểm tra tensor, tính norm và thống kê.
- `os`: kiểm tra file tồn tại, lấy tên file từ đường dẫn.
- `numpy`: import sẵn (trong code hiện tại **chưa sử dụng**).

## Cấu hình đường dẫn file
- `FILE_PATH`: đường dẫn tới file `.pt` cần phân tích.

Ví dụ:
- `/content/drive/.../begin_2/sentence_0_ARG_0.pt`

> Lưu ý: nếu chạy trên Google Colab, hãy chắc chắn đã mount Google Drive để đường dẫn `/content/drive/...` tồn tại.


## Hàm `print_tensor_stats(name, tensor)`

### Mục đích
In ra thống kê chi tiết cho một tensor, ứng với một `key` trong dict.

### Logic chính
- Đưa tensor về CPU và float:
  - `t = tensor.detach().cpu().float()`
- In thông tin:
  - **KEY**: tên tensor
  - **Shape**: kích thước tensor
  - **Type**: dtype tensor

### Thống kê cơ bản
Tính và in:
- `Min`, `Max`, `Mean`, `Std`

Mục đích:
- kiểm tra phân phối giá trị của embedding/weights
- phát hiện bất thường (quá lớn, quá nhỏ, std gần 0, v.v.)

### L2 Norm
- `l2_norm = torch.norm(t, p=2)`

Mục đích:
- đo “độ dài” vector
- hữu ích khi so sánh embedding (cosine / Euclid)

### Hiển thị dữ liệu mẫu
- Làm phẳng tensor:
  - `flat_t = t.flatten()`

#### Trường hợp key chứa `"token_weights"`
- In toàn bộ danh sách trọng số token:
  - `flat_t.tolist()`
- In tổng trọng số:
  - `flat_t.sum()`
  - thường dùng để kiểm tra attention-like weights (hay tổng gần 1)

#### Trường hợp embedding/vector dài
- In 5 giá trị đầu và 5 giá trị cuối:
  - `head = flat_t[:5]`
  - `tail = flat_t[-5:]`

### Kiểm tra sparsity (phần tử bằng 0)
- Đếm số phần tử bằng 0:
  - `zero_count = (flat_t == 0).sum().item()`
- Nếu `zero_count > 0`:
  - in ra số lượng và phần trăm phần tử bằng 0

## Hàm `inspect_pt_file(file_path)`

### Mục đích
- Kiểm tra file tồn tại
- Load dữ liệu bằng `torch.load`
- Nếu là dict → duyệt từng key để in thống kê
- Nếu là tensor đơn → in thống kê tensor đó

### Bước 1 — In thông tin file đang phân tích
- In tên file: `os.path.basename(file_path)`
- In full path để debug

### Bước 2 — Kiểm tra file tồn tại
- Nếu không tồn tại ở đường dẫn đã cho:
  - thử tìm file cùng tên ở thư mục hiện tại (local)
- Nếu vẫn không tồn tại → dừng và báo lỗi

### Bước 3 — Load file `.pt`
- Dùng:
  - `torch.load(file_path, map_location=torch.device('cpu'))`
- `map_location='cpu'` đảm bảo load được cả file lưu từ GPU

## Xử lý dữ liệu sau khi load

### Trường hợp 1: Dữ liệu là `dict`
- In `Tổng số Keys`
- Sắp xếp keys để hiển thị theo thứ tự layer (tránh `layer_10` đứng trước `layer_2`)
- Với mỗi `key`:
  - nếu value là tensor → gọi `print_tensor_stats(key, value)`
  - nếu không phải tensor → in trực tiếp (metadata hoặc giá trị khác)

### Trường hợp 2: Dữ liệu là tensor đơn
- Gọi:
  - `print_tensor_stats("Tensor duy nhất", data)`

### Trường hợp 3: Dữ liệu không phải dict/tensor
- In thông báo dữ liệu không đúng định dạng mong đợi

## Bắt lỗi khi đọc file
Toàn bộ quá trình load/parse được bọc trong `try/except` để:
- tránh crash khi file hỏng / sai định dạng / khác version torch
- in ra lỗi cụ thể để debug


## Khối chạy chính (`__main__`)
Khi chạy file như script:
- `inspect_pt_file(FILE_PATH)` sẽ được gọi
- output sẽ in đầy đủ thống kê theo cấu trúc `.pt`

## Kết quả đầu ra
Kết quả in ra console sẽ cho bạn:
- File `.pt` chứa những key nào
- Mỗi tensor có shape/dtype gì
- Thống kê giá trị (min/max/mean/std)
- L2 norm của vector
- Mẫu dữ liệu để đối chiếu nhanh
- (tuỳ trường hợp) danh sách `token_weights` + tổng trọng số

In [ ]:
import torch
import os
import numpy as np

# Đường dẫn file (Bạn có thể thay đổi đường dẫn này)
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_test_gramvar_inner_content/begin_2/sentence_0_ARG_0.pt"

def print_tensor_stats(name, tensor):
    """Hàm phụ trợ để in thông tin chi tiết của một tensor"""
    # Chuyển về CPU và float để tính toán thống kê dễ dàng
    t = tensor.detach().cpu().float()

    print(f"🔹 KEY: '{name}'")
    print(f"   • Shape: {list(t.shape)}")
    print(f"   • Type:  {t.dtype}")

    # 1. Thống kê cơ bản (Min, Max, Mean, Std)
    print(f"   • Stats: Min={t.min():.4f} | Max={t.max():.4f} | Mean={t.mean():.4f} | Std={t.std():.4f}")

    # 2. L2 Norm (Độ dài vector - quan trọng cho việc so sánh khoảng cách)
    l2_norm = torch.norm(t, p=2)
    print(f"   • L2 Norm: {l2_norm:.4f}")

    # 3. Hiển thị dữ liệu mẫu
    flat_t = t.flatten()

    # Nếu là Token Weights (thường số lượng phần tử ít, bằng số token trong span)
    if "token_weights" in name:
        print(f"   • VALUES (Trọng số token): {flat_t.tolist()}")
        # Kiểm tra tổng trọng số (thường nên xấp xỉ 1 nếu là attention weights, hoặc >0)
        print(f"   • Sum Weights: {flat_t.sum():.4f}")

    # Nếu là Vector đặc trưng (Embedding - thường rất dài, ví dụ 2304)
    else:
        # In 5 giá trị đầu và 5 giá trị cuối
        head = [round(x, 4) for x in flat_t[:5].tolist()]
        tail = [round(x, 4) for x in flat_t[-5:].tolist()]
        print(f"   • SAMPLE: {head} ... {tail}")

        # Kiểm tra số lượng giá trị bằng 0 (Sparsity check)
        zero_count = (flat_t == 0).sum().item()
        if zero_count > 0:
            print(f"   • Sparsity: Có {zero_count} phần tử bằng 0 ({zero_count/len(flat_t)*100:.2f}%)")

    print("-" * 50)

def inspect_pt_file(file_path):
    print(f"--- PHÂN TÍCH CHI TIẾT FILE: {os.path.basename(file_path)} ---")
    print(f"Path: {file_path}\n")

    if not os.path.exists(file_path):
        print(f"❌ Lỗi: Không tìm thấy file.")
        # Thử tìm ở thư mục hiện tại nếu đường dẫn tuyệt đối sai
        local_path = os.path.basename(file_path)
        if os.path.exists(local_path):
            print(f"-> Đã tìm thấy file local: {local_path}")
            file_path = local_path
        else:
            return

    try:
        data = torch.load(file_path, map_location=torch.device('cpu'))

        if isinstance(data, dict):
            print(f"Tổng số Keys: {len(data)}")
            print("=" * 50)

            # Sắp xếp keys để in theo thứ tự layer 0 -> 12
            # Hàm sort key giúp layer_2 đứng trước layer_10
            sorted_keys = sorted(data.keys(), key=lambda x: (x.split('_')[0], int(x.split('_')[1]) if x.split('_')[1].isdigit() else 999))

            for key in sorted_keys:
                value = data[key]
                if torch.is_tensor(value):
                    print_tensor_stats(key, value)
                else:
                    print(f"🔹 KEY: '{key}' | Value: {value}")
                    print("-" * 50)

        elif torch.is_tensor(data):
            print_tensor_stats("Tensor duy nhất", data)
        else:
            print(f"Dữ liệu không phải Dict hay Tensor: {data}")

    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")

if __name__ == "__main__":
    inspect_pt_file(FILE_PATH)

--- PHÂN TÍCH CHI TIẾT FILE: sentence_0_ARG_0.pt ---
Path: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_test_gramvar_inner_content/begin_2/sentence_0_ARG_0.pt

Tổng số Keys: 26
🔹 KEY: 'layer_0'
   • Shape: [2304]
   • Type:  torch.float32
   • Stats: Min=-0.2074 | Max=0.2487 | Mean=0.0002 | Std=0.0454
   • L2 Norm: 2.1774
   • SAMPLE: [-0.0904, -0.0152, 0.1491, -0.1204, 0.0173] ... [0.0229, -0.0039, -0.0418, 0.0067, -0.0459]
--------------------------------------------------
🔹 KEY: 'layer_0_token_weights'
   • Shape: [13]
   • Type:  torch.float32
   • Stats: Min=0.0000 | Max=0.1590 | Mean=0.0405 | Std=0.0530
   • L2 Norm: 0.2344
   • VALUES (Trọng số token): [0.0, 0.04448377713561058, 0.0, 0.0, 0.0, 0.12197980284690857, 0.10046704858541489, 0.02085447683930397, 0.0345047302544117, 0.1589740663766861, 0.01239015068858862, 0.0018204599618911743, 0.030603699386119843]
   • Sum Weights: 0.5261
--------------------------------------------------
🔹 KEY:

In [ ]:
import torch
import os

# Đường dẫn đến file bạn vừa tải lên
# Nếu chạy trên Colab, hãy đảm bảo bạn đã upload file này lên
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Ablation_Study_Data/Span Adaptation Vector/Baseline E/Train/span_regular_vectors_train_gramvar_baseline_E_inner_content/lead/sentence_0_ARG_1.pt"

def inspect_pt_file(file_path):
    print(f"--- ĐANG ĐỌC FILE: {file_path} ---")

    if not os.path.exists(file_path):
        print(f"❌ Lỗi: Không tìm thấy file tại '{file_path}'")
        return

    try:
        # Tải file .pt (map_location='cpu' để đảm bảo chạy được ngay cả khi không có GPU)
        data = torch.load(file_path, map_location=torch.device('cpu'))

        print(f"✅ Tải thành công!")
        print(f"Loại dữ liệu (Type): {type(data)}")

        # Trường hợp 1: Dữ liệu là Dictionary (thường gặp nhất trong project của bạn)
        if isinstance(data, dict):
            print(f"\n--- CẤU TRÚC DỮ LIỆU ---")
            print(f"Tổng số keys: {len(data)}")

            for key, value in data.items():
                if torch.is_tensor(value):
                    print(f"Key: '{key:<25}' | Shape: {list(value.shape)} | Type: Tensor")
                    # In thử 5 giá trị đầu tiên của layer đầu tiên tìm thấy để xem mẫu
                    if "layer_0" in key:
                         print(f"   -> Mẫu dữ liệu (5 số đầu): {value.flatten()[:5].tolist()}")
                else:
                    print(f"Key: '{key:<25}' | Value: {value}")

        # Trường hợp 2: Dữ liệu là Tensor đơn lẻ
        elif torch.is_tensor(data):
            print(f"\n--- DỮ LIỆU LÀ TENSOR ---")
            print(f"Shape: {data.shape}")
            print(f"Dữ liệu mẫu: {data.flatten()[:10]}...")

        else:
            print(f"Dữ liệu bên trong là: {data}")

    except Exception as e:
        print(f"❌ Lỗi khi đọc file bằng torch: {e}")
        print("Gợi ý: Hãy chắc chắn bạn đã cài pytorch (pip install torch)")

if __name__ == "__main__":
    inspect_pt_file(FILE_PATH)

--- ĐANG ĐỌC FILE: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Ablation_Study_Data/Span Adaptation Vector/Baseline E/Train/span_regular_vectors_train_gramvar_baseline_E_inner_content/lead/sentence_0_ARG_1.pt ---
✅ Tải thành công!
Loại dữ liệu (Type): <class 'dict'>

--- CẤU TRÚC DỮ LIỆU ---
Tổng số keys: 26
Key: 'layer_0                  ' | Shape: [2304] | Type: Tensor
   -> Mẫu dữ liệu (5 số đầu): [-0.21990421414375305, 0.15868109464645386, 1.23786199092865, -0.5026942491531372, 0.7339382171630859]
Key: 'layer_0_token_weights    ' | Shape: [3] | Type: Tensor
   -> Mẫu dữ liệu (5 số đầu): [0.3333333432674408, 0.3333333432674408, 0.3333333432674408]
Key: 'layer_1                  ' | Shape: [2304] | Type: Tensor
Key: 'layer_1_token_weights    ' | Shape: [3] | Type: Tensor
Key: 'layer_2                  ' | Shape: [2304] | Type: Tensor
Key: 'layer_2_token_weights    ' | Shape: [3] | Type: Tensor
Key: 'layer_3                  ' | Shape: [2304] | Type: Tensor
Key: 'layer_